In [88]:
import os
import csv
from csv import reader
import random

max_seq_length = 128
mysql_path = '../data/def_use/out/mysql_index.tsv'
outpath = '../data/def_use/mysql/'

In [89]:
def convertString(use):
  use_list = list(map(int, use.split()))
  pos_list = [u for u in use_list if (u >= 0 and u <= max_seq_length)]
  if len(pos_list) > 0:
    return ' '.join(map(str, pos_list))
  else:
    return 'None'

In [90]:
def shuffle_and_divide(fpath, outpath):
  if not os.path.exists(os.path.dirname(outpath)):
    try:
      os.makedirs(os.path.dirname(outpath))
    except OSError as exc:
      if exc.errno != errno.EEXIST:
        raise

  with open(outpath+'train.tsv', 'w', newline='') as f_train, open(outpath+'dev.tsv', 'w', newline='') as f_dev, open(outpath+'test.tsv','w', newline='') as f_test:
      tsv_train = csv.writer(f_train, delimiter='\t')
      tsv_dev = csv.writer(f_dev, delimiter='\t')
      tsv_test = csv.writer(f_test, delimiter='\t')

      tsv_train.writerow(['vocab_val_seq', 'def1_index', 'use1_index', 'def2_index', 'use2_index', 'label'])
      tsv_dev.writerow(['vocab_val_seq', 'def1_index', 'use1_index', 'def2_index', 'use2_index', 'label'])
      tsv_test.writerow(['vocab_val_seq', 'def1_index', 'use1_index', 'def2_index', 'use2_index', 'label'])

      fpath_shuffled = os.path.splitext(fpath)[0] + '_shuffled.tsv'
      with open(fpath, 'r') as f_read:
        data = f_read.readlines()
        header, rows = data[0], data[1:]
        filtered_rows = []
        for row in rows:
          line = row.strip().split("\t")
          text = list(map(int, line[1].split()))[:max_seq_length]
          def1 = line[2]
          def2 = line[4]
          use1 = convertString(line[3])
          use2 = convertString(line[5])
          label = line[6]
          if (int(def1) < max_seq_length and int(def2) < max_seq_length):
            filtered_rows.append([' '.join(map(str, text)), def1, use1, def2, use2, label])
        random.shuffle(filtered_rows)
        print(len(filtered_rows))
              
      with open(outpath+'test_labels.txt' ,'w') as f_labels:
        i = 0
        for row in filtered_rows:
          if i <= 0.7 * len(filtered_rows):
            tsv_train.writerow(row)
          elif i <= 0.85 * len(filtered_rows):
            tsv_dev.writerow(row)
          else:
            tsv_test.writerow(row)
            f_labels.write(row[-1] + ' ')
          i += 1

In [91]:
shuffle_and_divide(mysql_path, outpath)

479
